In [240]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import re
import csv

root = "http://www.moel.go.kr/index.do"
web_link = list()
result = list()

# 고용노동부 내의 복지 페이지 링크를 모두 가져옴
def enum_links(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("#main_content > div.main_policy")
    herfs = re.findall('(?<=href=").*?(?=[\"])', str(links))
    
    # 사이트 페이지의 복지 링크 추가
    for herf in herfs:
        url = urljoin(root, herf)
        web_link.append(url)
    
    # 복지링크 내의 링크 탐색
    for i in range(len(web_link)):
        link = web_link.pop(0)
        html = requests.get(link).text
        soup = BeautifulSoup(html, "html.parser")
        discription = soup.select("#txt > ul > li > a")
        
        for dis in discription:
            dis = str(dis)
            url_ = urljoin(link, (re.findall('(?<=href=").*?(?=;)', dis)[0]))
            title = re.findall('.*?_bt[ ]?">.*?(?=<br/>|</p>)', dis)
            title = title[0][title[0].find('>')+1:] if len(title) else 'NULL'
            target = re.findall('대상.*?(?=<br/>|</p>)', dis)
            target = target[0][target[0].find('>')+1:] if len(target) else 'NULL'
            
            text = requests.get(url_).text
            text_soup = BeautifulSoup(text, "html.parser")
            text = text_soup.select("#txt > div.po_view.mt_20 > div.po_content") 
            if len(text)==0 : text = text_soup.select("#txt > div.po_content") 
            
            result.append([title, target, text, url_])

def save_file(text):
    csv_file = open("./moel.csv", "w", newline="", encoding='UTF8')
    csv_writer = csv.writer(csv_file)
    for row in text:
        csv_writer.writerow(row)
    csv_file.close()

In [241]:
if __name__ == "__main__":
    url = root 
    enum_links(url)
    save_file(result)